In [ ]:
# Copyright 2021 NVIDIA CORPORATION
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3" 

import matplotlib.pyplot as plt, time
import tensorflow as tf
import cudf, numpy as np, pandas as pd
pd.set_option('display.max_columns', 100)
import cupy, gc
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
tf.__version__

'2.3.0'

In [2]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [3]:
#tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
#print('Mixed precision enabled')

In [4]:
VER = 89
POSTFIX = 'a'

In [5]:
# ALL OF GIBA PREPROCESS FEATURES
ALL = ['hashtags', 'tweet_id', 'media', 'links', 'domains', 'tweet_type',
       'language', 'timestamp', 'a_user_id', 'a_follower_count',
       'a_following_count', 'a_is_verified', 'a_account_creation', 'b_user_id',
       'b_follower_count', 'b_following_count', 'b_is_verified',
       'b_account_creation', 'b_follows_a', 'reply', 'retweet',
       'retweet_comment', 'like', 'text', 'tw_len_media', 'tw_len_photo',
       'tw_len_video', 'tw_len_gif', 'tw_len_quest', 'tw_len_token',
       'tw_count_capital_words', 'tw_count_excl_quest_marks',
       'tw_count_special1', 'tw_count_hash', 'tw_last_quest', 'tw_len_retweet',
       'tw_len_rt', 'tw_count_at', 'tw_count_words', 'tw_count_char',
       'tw_rt_count_words', 'tw_rt_count_char', 'tw_original_user0',
       'tw_original_user1', 'tw_original_user2', 'tw_rt_user0',
       'tw_original_http0', 'tw_word0', 'tw_word1', 'tw_word2', 'tw_word3',
       'tw_word4', 'tw_tweet', 'group', 'dt_day', 'dt_dow', 'dt_minute',
       'len_hashtags', 'len_links', 'len_domains']

# Features

In [6]:
FEATS = ['b_follows_a','a_follower_count','a_following_count','b_follower_count','b_following_count'] #,
#         'a_account_creation','b_account_creation',
#         'a_is_verified','b_is_verified']

FEATS += ['dt_dow', 'dt_minute',
          'len_hashtags', 'len_links', 'len_domains']

FEATS += ['tw_len_media', 'tw_len_photo','tw_len_video', 'tw_len_gif', 
          'tw_len_quest', 'tw_len_token',
          'tw_count_capital_words', 'tw_count_excl_quest_marks',
          'tw_count_special1', 'tw_count_hash', 'tw_last_quest', 
          'tw_len_retweet', 'tw_len_rt', 'tw_count_at', 'tw_count_words', 'tw_count_char',
          'tw_rt_count_words', 'tw_rt_count_char']

FEATS2 = ['a_user_id', 'b_user_id','tweet_type','language','media']

# Load Train Valid
If columns in TE2 are not present in FEATS and TE, then add below

In [7]:
TARS = ['reply', 'retweet','retweet_comment', 'like']
COLS = TARS + FEATS2 + FEATS 

In [8]:
train = pd.read_parquet(f'/raid/RecSys/recsys2021/parquet0/train12{POSTFIX}.parquet',columns=COLS).reset_index(drop=True)
print( train.shape )
train.head()

(31977926, 37)


,reply,retweet,retweet_comment,like,a_user_id,b_user_id,tweet_type,language,media,b_follows_a,a_follower_count,a_following_count,b_follower_count,b_following_count,dt_dow,dt_minute,len_hashtags,len_links,len_domains,tw_len_media,tw_len_photo,tw_len_video,tw_len_gif,tw_len_quest,tw_len_token,tw_count_capital_words,tw_count_excl_quest_marks,tw_count_special1,tw_count_hash,tw_last_quest,tw_len_retweet,tw_len_rt,tw_count_at,tw_count_words,tw_count_char,tw_rt_count_words,tw_rt_count_char
0,0,0,0,0,-738100672,1762493187,0,10,4,0,5785019,818,38,75,4,827,0,0,0,1,0,0,1,0,43,9,0,6,0,0,0,0,0,13,57,0,0
1,0,0,0,0,768981309,-118189451,0,7,0,0,20228,7197,338,513,3,1022,0,0,0,0,0,0,0,0,18,3,0,0,0,0,0,0,0,5,41,0,0
2,0,0,0,1,1114089211,682509122,0,0,0,0,433964,1832,1205,775,1,1025,0,0,0,0,0,0,0,0,8,2,0,0,0,0,0,0,0,5,27,0,0
3,0,0,0,1,1051751361,-238649212,2,4,2,1,344,335,87,104,3,1176,0,0,0,3,3,0,0,0,18,3,0,0,0,0,0,0,0,1,20,0,0
4,0,0,0,0,-1515818908,1750659970,0,0,1,0,10926,349,4120,5004,3,1095,0,0,0,1,1,0,0,0,28,2,0,0,0,0,0,0,0,8,61,0,0


In [9]:
valid = pd.read_parquet(f'/raid/RecSys/recsys2021/parquet0/valid12{POSTFIX}.parquet',columns=COLS).reset_index(drop=True)
print( valid.shape )
valid.head()

(17296496, 37)


,reply,retweet,retweet_comment,like,a_user_id,b_user_id,tweet_type,language,media,b_follows_a,a_follower_count,a_following_count,b_follower_count,b_following_count,dt_dow,dt_minute,len_hashtags,len_links,len_domains,tw_len_media,tw_len_photo,tw_len_video,tw_len_gif,tw_len_quest,tw_len_token,tw_count_capital_words,tw_count_excl_quest_marks,tw_count_special1,tw_count_hash,tw_last_quest,tw_len_retweet,tw_len_rt,tw_count_at,tw_count_words,tw_count_char,tw_rt_count_words,tw_rt_count_char
0,0,0,0,1,-1720532695,-737683714,0,1,0,1,294,531,523,681,6,911,1,0,0,0,0,0,0,0,72,3,0,2,1,0,0,0,0,36,139,0,0
1,0,0,0,1,-940870246,617034320,0,5,0,0,147068,195,247,231,0,1143,1,0,0,0,0,0,0,0,101,7,0,2,1,0,0,0,0,48,299,0,0
2,0,0,0,0,-1434746634,-1464478894,1,0,0,1,5754,1750,3209,770,5,1100,1,0,0,0,0,0,0,0,45,4,0,0,1,0,0,0,3,18,131,0,8
3,0,0,0,1,894603071,1776705041,0,1,0,0,5675,602,598,859,6,642,0,1,1,0,0,0,0,0,59,2,0,0,0,0,0,0,0,29,85,0,0
4,0,0,0,1,119885682,1484342501,0,0,0,0,571168,151,270,117,4,1206,0,0,0,0,0,0,0,0,81,2,0,0,0,0,0,0,0,63,288,0,0


# Process Tokens

In [10]:
%%time
train_tokens = np.load(f'/raid/RecSys/recsys2021/parquet0/train_tokens_48_{POSTFIX}.npy')
#train_tokens = np.zeros((len(train),32),dtype='int32')
#for k in range(len(train)):
#    if k%100_000==0: print(k,', ',end='')
#    x = np.array( train.iloc[k,0].split('\t')[:32] ).astype('int32')
#    train_tokens[k,:len(x)] = x

CPU times: user 21.7 ms, sys: 2.8 s, total: 2.83 s
Wall time: 3.87 s


In [11]:
%%time
valid_tokens = np.load(f'/raid/RecSys/recsys2021/parquet0/valid_tokens_48_{POSTFIX}.npy')
#valid_tokens = np.zeros((len(valid),32),dtype='int32')
#for k in range(len(valid)):
#    if k%100_000==0: print(k,', ',end='')
#    x = np.array( valid.iloc[k,0].split('\t')[:32] ).astype('int32')
#    valid_tokens[k,:len(x)] = x

CPU times: user 15 ms, sys: 1.15 s, total: 1.16 s
Wall time: 2.07 s


# Process Features

In [12]:
TARGETS = ['reply','retweet','retweet_comment','like']

In [13]:
LOG_FEATS = ['a_follower_count','a_following_count','b_follower_count','b_following_count']

In [14]:
NUM_FEATS1 = []
for f in LOG_FEATS:
    name = 'LOG_'+f
    print(name)
    train[name] = np.log1p( train[f].values, dtype='float32' )
    valid[name] = np.log1p( valid[f].values, dtype='float32' )
    NUM_FEATS1.append( name )

LOG_a_follower_count
LOG_a_following_count
LOG_b_follower_count
LOG_b_following_count


In [15]:
NORM_FEATS = ['LOG_a_follower_count','LOG_a_following_count',
              'dt_minute','len_hashtags','len_links','len_domains']
              #'a_account_creation','b_account_creation','a_is_verified','b_is_verified']

NORM_FEATS += ['tw_len_media', 'tw_len_photo','tw_len_video', 'tw_len_gif', 
          'tw_len_quest', 'tw_len_token',
          'tw_count_capital_words', 'tw_count_excl_quest_marks',
          'tw_count_special1', 'tw_count_hash', 'tw_last_quest', 
          'tw_len_retweet', 'tw_len_rt', 'tw_count_at', 'tw_count_words', 'tw_count_char',
          'tw_rt_count_words', 'tw_rt_count_char']

NORM_FEATS += ['LOG_b_follower_count','LOG_b_following_count','b_follows_a']

In [16]:
rows = []

NUM_FEATS2 = []
for f in NORM_FEATS:
    name = 'NORM_'+f
    print(name)
    mn = train[f].mean()
    st = train[f].std()
    dd = {}
    dd['feature'] = f
    dd['mean'] = mn
    dd['std'] = st
    rows.append(dd)
    train[name] = ((train[f].values - mn) /st).astype('float32')
    valid[name] = ((valid[f].values - mn) /st).astype('float32')
    NUM_FEATS2.append( name )
    
standardize = pd.DataFrame(rows)

NORM_LOG_a_follower_count
NORM_LOG_a_following_count
NORM_dt_minute
NORM_len_hashtags
NORM_len_links
NORM_len_domains
NORM_tw_len_media
NORM_tw_len_photo
NORM_tw_len_video
NORM_tw_len_gif
NORM_tw_len_quest
NORM_tw_len_token
NORM_tw_count_capital_words
NORM_tw_count_excl_quest_marks
NORM_tw_count_special1
NORM_tw_count_hash
NORM_tw_last_quest
NORM_tw_len_retweet
NORM_tw_len_rt
NORM_tw_count_at
NORM_tw_count_words
NORM_tw_count_char
NORM_tw_rt_count_words
NORM_tw_rt_count_char
NORM_LOG_b_follower_count
NORM_LOG_b_following_count
NORM_b_follows_a


In [17]:
standardize.to_csv('standardize_%i.csv'%VER,index=False)
standardize.head()

,feature,mean,std
0,LOG_a_follower_count,9.609736,3.075770
1,LOG_a_following_count,6.323446,1.902467
2,dt_minute,748.958455,405.572885
3,len_hashtags,0.421623,1.158650
4,len_links,0.132951,0.365483


# Embed Maps

In [18]:
FILTER = 5

In [19]:
tmp = train[['a_user_id']].copy()
vc = tmp.a_user_id.value_counts()
IDX_A = vc.loc[vc>FILTER].index.values

In [20]:
tmp = train[['b_user_id']].copy()
vc = tmp.b_user_id.value_counts()
IDX_B = vc.loc[vc>FILTER].index.values

In [21]:
IDX = np.union1d(IDX_A,IDX_B)
len(IDX)

1853679

In [22]:
IDX[:5]

array([-2147473346, -2147472674, -2147471436, -2147468482, -2147467862])

In [23]:
user_map = {x:y for x,y in zip(IDX,1+np.arange(len(IDX)))}

In [24]:
np.save('user_map_%i'%VER,IDX)

In [25]:
del tmp, vc #, v, code
gc.collect()

180

In [26]:
train['a_user_id'] = train.a_user_id.map(user_map).fillna(0).astype('int32')
valid['a_user_id'] = valid.a_user_id.map(user_map).fillna(0).astype('int32')

In [27]:
train['b_user_id'] = train.b_user_id.map(user_map).fillna(0).astype('int32')
valid['b_user_id'] = valid.b_user_id.map(user_map).fillna(0).astype('int32')

In [28]:
CAT_FEATS = ['a_user_id','b_user_id','tweet_type','language','media','dt_dow']

In [29]:
for f in CAT_FEATS:
    m = train[f].max()
    print(f,m)

a_user_id 1853674
b_user_id 1853679
tweet_type 2
language 65
media 13
dt_dow 6


# Features

In [30]:
FEATURES = CAT_FEATS + NUM_FEATS2
np.asarray( FEATURES )

array(['a_user_id', 'b_user_id', 'tweet_type', 'language', 'media',
       'dt_dow', 'NORM_LOG_a_follower_count',
       'NORM_LOG_a_following_count', 'NORM_dt_minute',
       'NORM_len_hashtags', 'NORM_len_links', 'NORM_len_domains',
       'NORM_tw_len_media', 'NORM_tw_len_photo', 'NORM_tw_len_video',
       'NORM_tw_len_gif', 'NORM_tw_len_quest', 'NORM_tw_len_token',
       'NORM_tw_count_capital_words', 'NORM_tw_count_excl_quest_marks',
       'NORM_tw_count_special1', 'NORM_tw_count_hash',
       'NORM_tw_last_quest', 'NORM_tw_len_retweet', 'NORM_tw_len_rt',
       'NORM_tw_count_at', 'NORM_tw_count_words', 'NORM_tw_count_char',
       'NORM_tw_rt_count_words', 'NORM_tw_rt_count_char',
       'NORM_LOG_b_follower_count', 'NORM_LOG_b_following_count',
       'NORM_b_follows_a'], dtype='<U30')

# NN Model

In [31]:
EMB_SIZE = 96
EMB_SIZE2 = 96*2

def build_model():
    inp = tf.keras.layers.Input(shape=(len(FEATURES),))
    inp2 = tf.keras.layers.Input(shape=(48,))

    embeds = []
    embeds.append( tf.keras.layers.Embedding(len(IDX)+1,EMB_SIZE) ) # USER_ID
    
    embeds.append( tf.keras.layers.Embedding(3,2) ) # TWEET_TYPE
    embeds.append( tf.keras.layers.Embedding(66,10) ) # LANGUAGE
    embeds.append( tf.keras.layers.Embedding(14,4) ) # MEDIA
    embeds.append( tf.keras.layers.Embedding(7,4) ) # DT_DOW
       
    # USERS
    a_user = embeds[0](inp[:,0])
    b_user = embeds[0](inp[:,1])
    
    # USER INTERACTION
    a_embed = tf.keras.layers.Concatenate()([a_user,inp[:,-len(NORM_FEATS):-len(NORM_FEATS)+2],inp[:,-1:]])
    a_embed = tf.keras.layers.Dense(EMB_SIZE,activation='tanh')(a_embed)
    a_dot_b = tf.keras.layers.Dot(axes=-1,normalize=True)([a_embed,b_user])
        
    # CAT FEATURE EMBEDDINGS
    embeds2 = []    
    for k in range(2,len(CAT_FEATS)):
        embeds2.append( embeds[k-1](inp[:,k]) )
    x1 = tf.keras.layers.Concatenate()(embeds2)
        
    # TWEET TOKEN EMBEDDINGS
    embeds3 = []
    word_emb = tf.keras.layers.Embedding(119548,EMB_SIZE2)
    for k in range(48):
        embeds3.append( word_emb(inp2[:,k]) )
    x2 = tf.keras.layers.Average()(embeds3)
    
    # USER INTERACT WITH TWEET
    tweet_embed = tf.keras.layers.Concatenate()([x1,x2,inp[:,-len(NORM_FEATS):-3]])
    tweet_embed = tf.keras.layers.Dense(EMB_SIZE,activation='tanh')(tweet_embed)
    #b_user_embed = tf.keras.layers.Dense(EMB_SIZE2,activation='tanh')(b_user)
    b_dot_tweet = tf.keras.layers.Dot(axes=-1,normalize=True)([tweet_embed,b_user])

    # NUMERICAL FEATURES
    x = tf.keras.layers.Concatenate()(
        [a_user,b_user,a_dot_b,b_dot_tweet,x1,x2,inp[:,-len(NORM_FEATS):]])
    
    HIDDEN_SIZE = 256 + 64
    LAYERS = 3
    
    for k in range(LAYERS):
        x = tf.keras.layers.Dense(HIDDEN_SIZE)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
        
    # CONCAT WITH MATRIX FACTORIZATION
    x = tf.keras.layers.Concatenate()([a_dot_b,b_dot_tweet,x])
    
    x = tf.keras.layers.Dense(4,activation='sigmoid',dtype='float32')(x)
    model = tf.keras.models.Model(inputs=[inp,inp2],outputs=x)
    
    opt = tf.keras.optimizers.Adam(lr=1e-3)
    loss = tf.keras.losses.BinaryCrossentropy()
    model.compile(loss=loss, optimizer = opt)
    return model

In [32]:
from sklearn.metrics import precision_recall_curve, auc, log_loss, average_precision_score

def compute_prauc(pred, gt):
  prec, recall, thresh = precision_recall_curve(gt, pred)
  prauc = auc(recall, prec)
  return prauc

def calculate_ctr(gt):
  positive = len([x for x in gt if x == 1])
  ctr = positive/float(len(gt))
  return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

# FAST METRIC FROM GIBA
def compute_rce_fast(pred, gt):
    cross_entropy = log_loss(gt, pred)
    yt = np.mean(gt)     
    strawman_cross_entropy = -(yt*np.log(yt) + (1 - yt)*np.log(1 - yt))
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

In [33]:
with strategy.scope():
    model = build_model()
    
model.fit([train[FEATURES],train_tokens],train[TARGETS],
          validation_data = ([valid[FEATURES],valid_tokens],valid[TARGETS]),
          epochs=1, verbose=1, batch_size=1024*8) 

oof = model.predict([valid[FEATURES],valid_tokens], verbose=1, batch_size=1024*8)

if 1:
    txt = ''; tot=0
    for i in range(4):
        #prauc = compute_prauc(oof[:,i], valid[TARGETS[i]].values )
        ap = average_precision_score( valid[TARGETS[i]].values, oof[:,i] )
        rce   = compute_rce_fast(oof[:,i], valid[TARGETS[i]].values )
        txt_ = f"{TARGETS[i]:20} AP:{ap:.5f} RCE:{rce:.5f}"
        print(txt_)
        txt += txt_ + '\n'
        tot += rce
    print()
    print('Total = %.5f'%tot)
    print()
    
np.save('nn%i'%VER,oof)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:batch_all_reduce: 18 all-reduces with algorithm = nccl, num_pa